<a href="https://colab.research.google.com/github/diogoalmeida1991/Anti-plagio/blob/main/anti_cola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import os
import re

In [13]:
#Cria o vetor de arquivos .csv
csv = []
for arquivo in os.listdir():
  if(re.search(".csv", arquivo)!=None):
    csv.append(arquivo)
csv.sort()

In [15]:
#Cria um vetor de dataframes, onde cada elemento representa uma atividade realizada
#Os csvs são criados com um script powershell com as seguintes colunas nome do
#arquivo, caminho e hash.
colas = []
for arquivo in csv:
  teste = pd.read_csv(arquivo)
  teste = teste.dropna()
  colas.append(teste[teste.duplicated(subset="Hash", keep=False)].copy()) #Coleta as linhas com hash igual
for df in colas:
  df["Nome"] = df["Caminho"].str.extract(r'\\([a-zA-Z?]+ \D+)_')#Coleta o nome do aluno do caminho
  df.drop("Caminho", axis=1, inplace=True)#Remove a coluna caminho
  df.drop_duplicates(subset = ['Nome', 'Hash'], inplace=True)#Remove envios duplicados


In [16]:
#Escreve os dados em uma planilha do excel os dados
ponteiro_linha = 0
with pd.ExcelWriter('Relatório.xlsx', engine='xlsxwriter') as writer:
  workbook = writer.book #Cria uma instância para o objeto
  worksheet = workbook.add_worksheet('Aba1')#Cria uma folha no excel.
  writer.sheets['Aba1'] = worksheet
  for i, df in enumerate(colas):
    worksheet.write(ponteiro_linha, 0, csv[i])#Escreve o nome do arquivo antes da tabela(linha, coluna, conteúdo)
    ponteiro_linha+=1
    df.to_excel(writer, sheet_name='Aba1', startrow=ponteiro_linha, index=False)#Grava o dataframe no excel.
    ponteiro_linha+=len(df) + 2
  #A Unidade de medida para largura no excel é baseada na largura do caractere "0" (zero) na fonte padrão Calibri, tamanho 11.
  worksheet.set_column(0, 0, 67) #Ajusta a coluna A para 67
  worksheet.set_column(1, 1, 70) #Ajusta a coluna B para 70
